In [3]:
import gzip
import yargy
import yargy.predicates as pred
import yargy.interpretation as interp
import yargy.morph

# Создаем Person
Person = interp.fact(
    'Person',
    ['name', 'birth_date', 'birth_place']
)

NAME_SEP = yargy.or_(pred.normalized(','), pred.normalized('-'))
NAME = yargy.or_(
    yargy.and_(pred.gram('Name'), yargy.not_(pred.gram('Abbr')), pred.is_capitalized()),
)

SURE = yargy.and_(pred.gram('Surn'), yargy.not_(pred.gram('Abbr')), pred.is_capitalized())
PART = yargy.and_(pred.gram('Patr'), yargy.not_(pred.gram('Abbr')), pred.is_capitalized())

# Правило для извлечения имени:
PERSON_NAME = yargy.rule(
    yargy.or_(
        yargy.rule(NAME, NAME_SEP.optional(), SURE.optional(), NAME_SEP.optional(), PART.optional(), NAME_SEP.optional()),
        yargy.rule(SURE, NAME_SEP.optional()),
        yargy.rule(PART, NAME_SEP.optional())
    )
)

In [4]:
SEP = yargy.or_(pred.normalized('.'), pred.normalized('/'), pred.normalized('-'))

DAY = yargy.and_(pred.gte(1), pred.lte(31))
MONTH = yargy.and_(pred.gte(1), pred.lte(12))
YEAR = yargy.and_(pred.gte(1), pred.lte(2025))

MONTHS = pred.dictionary([
    "января", "февраля", "марта", "апреля", "мая", "июня", "июля", "августа", "сентября", "октября",
    "ноября", "декабря", "январь", "февраль", "март", "апрель", "май", "июнь", "июль", "август",
    "сентябрь", "октябрь", "ноябрь", "декабрь", 
    "янв", "фев", "мар", "апр", "май", "июн", "июл", "авг", "сен", "окт", "ноя", "дек"
])

DATE_NUMERIC = yargy.rule(
    yargy.or_(
      yargy.rule(DAY, SEP, MONTH, SEP, YEAR),
      yargy.rule(YEAR, SEP, MONTH, SEP, DAY)
    )
)

DATE_TEXTUAL = yargy.rule(
    yargy.or_(
        yargy.rule(DAY, MONTHS, YEAR.optional()),
        yargy.rule(pred.normalized('в'), YEAR, pred.dictionary(['год', 'г.', 'году']))
    )
)

# Правило для извлечения даты рождения:
BIRTH_DATE = yargy.rule(
    pred.dictionary(['родился', 'родилась', 'рожден', 'рождена', 'дата рождения', 'дата рождения:']),
    yargy.or_(
        DATE_NUMERIC,
        DATE_TEXTUAL,
    ) 
)

In [5]:
BIRTH_PLACE_SEP = yargy.or_(pred.normalized(','), pred.normalized('.'))
PREP = pred.dictionary(["в", "на", "из", "во"])

# Правило для извлечения места рождения:
BIRTH_PLACE = yargy.rule(
    pred.dictionary(['родился', 'родилась', 'рожден', 'рождена']),
    PREP.optional(),
    yargy.or_(
        yargy.rule(pred.is_capitalized(), pred.normalized('-'), pred.is_capitalized(), BIRTH_PLACE_SEP.optional()),
        yargy.rule(pred.is_capitalized(), BIRTH_PLACE_SEP.optional())
    )
)

In [6]:
# Функция для работы форматом ГЗ
def parseFromGZ(file_path, parser):
    objects = []
    with gzip.open(file_path, 'rt', encoding='utf-8') as f:
        for line in f:
            matches = parser.findall(line)
            for match in matches:
                fact = match.fact
                objects.append(fact)
                print(fact)
    return objects

In [10]:
# Определяем правила для распознавания объекта PERSON 
PERSON = yargy.rule(
    PERSON_NAME.interpretation(Person.name),
    BIRTH_DATE.interpretation(Person.birth_date)
    ).interpretation(Person)

In [11]:
persons = parseFromGZ("C:/Users/MSI-PC/Desktop/news.txt.gz", yargy.Parser(PERSON))
persons

Person(name='Трэмиел', birth_date='родился в 1928 году', birth_place=None)
Person(name='Джордж,', birth_date='родился в 2013 году', birth_place=None)
Person(name='Третчиков,', birth_date='родившийся в 1913 году', birth_place=None)
Person(name='Смирнов', birth_date='родился 18 декабря 1937', birth_place=None)
Person(name='Картер', birth_date='родилась 7 января', birth_place=None)
Person(name='Николай Караченцов', birth_date='родился 27 октября 1944', birth_place=None)
Person(name='Сергей Довлатов', birth_date='родился в 1941 году', birth_place=None)
Person(name='Аня Титова', birth_date='родилась в 1984 году', birth_place=None)
Person(name='Джордж', birth_date='родился 22 июля 2013', birth_place=None)
Person(name='Игорь Доценко', birth_date='родился в 1953 году', birth_place=None)
Person(name='Андрей Сердюков', birth_date='родился в 1962 году', birth_place=None)
Person(name='Анатолий Сморгонский', birth_date='родился в 1979 году', birth_place=None)
Person(name='Энгельбарт', birth_date='р

[Person(
     name='Трэмиел',
     birth_date='родился в 1928 году',
     birth_place=None
 ),
 Person(
     name='Джордж,',
     birth_date='родился в 2013 году',
     birth_place=None
 ),
 Person(
     name='Третчиков,',
     birth_date='родившийся в 1913 году',
     birth_place=None
 ),
 Person(
     name='Смирнов',
     birth_date='родился 18 декабря 1937',
     birth_place=None
 ),
 Person(
     name='Картер',
     birth_date='родилась 7 января',
     birth_place=None
 ),
 Person(
     name='Николай Караченцов',
     birth_date='родился 27 октября 1944',
     birth_place=None
 ),
 Person(
     name='Сергей Довлатов',
     birth_date='родился в 1941 году',
     birth_place=None
 ),
 Person(
     name='Аня Титова',
     birth_date='родилась в 1984 году',
     birth_place=None
 ),
 Person(
     name='Джордж',
     birth_date='родился 22 июля 2013',
     birth_place=None
 ),
 Person(
     name='Игорь Доценко',
     birth_date='родился в 1953 году',
     birth_place=None
 ),
 Person(
 

In [9]:
# Определяем правила для распознавания объекта PERSON 
PERSON = yargy.rule(
    PERSON_NAME.interpretation(Person.name),
    BIRTH_PLACE.interpretation(Person.birth_place)
    ).interpretation(Person)

In [10]:
persons = parseFromGZ("C:/Users/MSI-PC/Desktop/news.txt.gz", yargy.Parser(PERSON))
persons

Person(name='Андрей Курносенко', birth_date=None, birth_place='родился в Севастополе')
Person(name='Гвасалия', birth_date=None, birth_place='родился в Грузии,')
Person(name='Руни,', birth_date=None, birth_place='Род Стюарт')
Person(name='ЯковлевЮрий Яковлев', birth_date=None, birth_place='родился в Москве')
Person(name='Патрик', birth_date=None, birth_place='родился в Бронксе')
Person(name='Кейру', birth_date=None, birth_place='родилась в Ростове')
Person(name='Инна Лиснянская', birth_date=None, birth_place='родилась в Баку')
Person(name='Николай Харджиев,', birth_date=None, birth_place='родившийся на Украине,')
Person(name='Терон,', birth_date=None, birth_place='родившаяся в ЮАР,')
Person(name='Гамильтон', birth_date=None, birth_place='родился в Лондоне')
Person(name='Джаррар', birth_date=None, birth_place='родилась в Каннах')
Person(name='Аламуддин', birth_date=None, birth_place='родилась в Ливане,')
Person(name='Рахлин', birth_date=None, birth_place='родился в Ленинграде')
Person(na

[Person(
     name='Андрей Курносенко',
     birth_date=None,
     birth_place='родился в Севастополе'
 ),
 Person(
     name='Гвасалия',
     birth_date=None,
     birth_place='родился в Грузии,'
 ),
 Person(
     name='Руни,',
     birth_date=None,
     birth_place='Род Стюарт'
 ),
 Person(
     name='ЯковлевЮрий Яковлев',
     birth_date=None,
     birth_place='родился в Москве'
 ),
 Person(
     name='Патрик',
     birth_date=None,
     birth_place='родился в Бронксе'
 ),
 Person(
     name='Кейру',
     birth_date=None,
     birth_place='родилась в Ростове'
 ),
 Person(
     name='Инна Лиснянская',
     birth_date=None,
     birth_place='родилась в Баку'
 ),
 Person(
     name='Николай Харджиев,',
     birth_date=None,
     birth_place='родившийся на Украине,'
 ),
 Person(
     name='Терон,',
     birth_date=None,
     birth_place='родившаяся в ЮАР,'
 ),
 Person(
     name='Гамильтон',
     birth_date=None,
     birth_place='родился в Лондоне'
 ),
 Person(
     name='Джаррар',
   